# Projeto 2 - Classificador Automático de Sentimento

Em três de setembro de 2018, os visionários engenheiros André Matarazzo e Felipe Kocinas projetavam o primeiro foguete capaz de chegar ao sol. Em meio a sua fama, sucesso e boa-vontade, a consagrada dupla foi contactada pela pizzaria Domino's, quem requisitou seus intelectos para analisar o desempenho da empresa a partir de comentários sobre seus produtos encontrados na internet.

A partir do conhecimento absoluto previamente adquirido na aula de ciência dos dados, a dupla recorreu ao Teorema de Bayes, classificando de forma magistral os comentários encontrados. Assim sendo, os aspirantes a vencedores do Nobel de Programação providenciaram um extraordinário relatório, que permitiu a Domino's a se tornar a pizzaria mais bem sucedida do mundo.

Em meio a tal inconfundível acontecimento, os CR 1 e 2 da faculdade Insper forneceram o passo-a-passo de seu projeto, com o intuíto de sempre passar adiante o conhecimento já dominado por eles.

___

## Parte I - Adquirindo a Base de Dados

In [ ]:
#imports
import pandas as pd
import math
#abrindo o arquivo do excel
Dominos = pd.read_excel("tweets_Dominos.xlsx")

___
## Parte II - Montando o Classificador Naive-Bayes


*código para limpar a pontuação e outros das frases*

In [2]:
def removepontuação(tweet):
    tweet = tweet.replace(",","")
    tweet = tweet.replace("!","")
    tweet = tweet.replace(".","")
    tweet = tweet.replace(";","")
    tweet = tweet.replace("?","")
    tweet = tweet.replace("(","")
    tweet = tweet.replace(")","")
    tweet = tweet.replace("@","")
    tweet = tweet.replace("+","")
    tweet = tweet.replace("'","")
    tweet = tweet.replace("-"," ")
    tweet = tweet.replace("...","")
    tweet = tweet.replace("#","")
    tweet = tweet.replace("\n","")
    tweet = tweet.replace("$","")
    tweet = tweet.replace("%","")
    tweet = tweet.replace("~","")
    tweet = tweet.replace("  "," ")
    tweet = tweet.replace("rt ","")
    tweet = tweet.replace("<","")
    tweet = tweet.replace(">","")
    tweet = tweet.replace(":","")
    tweet = tweet.replace("'","")
    tweet = tweet.replace("\'","")
    return tweet


*código para remoção de links*

In [3]:
def removepalavra(tweet):
    tweetsem = removepontuação(tweet)
    tweetlista = tweetsem.split(" ")
    i = 0
    while i<len(tweetlista):
            if len(tweetlista[i])>4:
                 if tweetlista[i][0] == "h" and tweetlista[i][1] == "t" and tweetlista[i][2] == "t":
                    del tweetlista[i]
            i+=1  
         
    return tweetlista

*lista com todos tweets do treinamento*

In [4]:
lista = []
for frase in Dominos.Treinamento:
    tweetslimpos1 = removepalavra(frase)
    lista.append(tweetslimpos1)

*juntando o que foi considerado positivo e negativo em uma lista de tweets relevantes*

In [ ]:
listarelevante = []
for frase in Dominos.Treinamento[Dominos.Característica == 0]:
        tweetslimpos = removepalavra(frase)
        if tweetslimpos not in listarelevante:
            listarelevante.append(tweetslimpos)

for frase in Dominos.Treinamento[Dominos.Característica == 1]:
        tweetslimpos = removepalavra(frase)
        if tweetslimpos not in listarelevante:
            listarelevante.append(tweetslimpos)

*contabilizando os tweets irrelevantes e também colocando-os em uma lista*

In [ ]:
listairrelevante = []
for frase in Dominos.Treinamento[Dominos.Característica == 2]:
        tweetslimpos2 = removepalavra(frase)
        if tweetslimpos not in listairrelevante:
            listairrelevante.append(tweetslimpos2)

*contando quantas vezes cada palavra aparece em sua suposta lista (relevante ou irrelevante) e colocando num dicionário para cada*

In [6]:
#dicionário relevante:
drel = {}
for frase in listarelevante:
    for palavra in frase:
        if palavra not in drel:
            drel[palavra]=1
        else:
            drel[palavra]+=1
            
#dicionário irrelevante:

dirrel = {}
for frase in listairrelevante:
    for palavra in frase:
        if palavra not in dirrel:
            dirrel[palavra]=1
        else:
            dirrel[palavra]+=1
            


*contando o total de palavras relevantes e irrelevantes*

In [ ]:
#palavras relevantes:
soma_palavras =  0
for palavra in drel:
    soma_palavras += drel[palavra]
    
    
#palavras irrelevantes:
soma_palavras1 =  0
for palavra in dirrel:
    soma_palavras1 += dirrel[palavra]
    
#soma_total:
soma_palavras_total = soma_palavras + soma_palavras1  

*calculando a probabilidade de ser relevante ou irrelevante*

In [41]:
contador0 = Dominos.Característica[Dominos.Característica == 0].count()
contador1 = Dominos.Característica[Dominos.Característica == 1].count()
contador2 = Dominos.Característica[Dominos.Característica == 2].count()
contador01 = contador0+contador1
contadorTotal = Dominos.Característica.count()


P_01 = (contador0+contador1)/contadorTotal 
print(contador01)
print(contador2)
P_2 = contador2/contadorTotal

print("Probabilidade de ser Relevante: {:.2f}%".format(P_01*100))
print("Probabilidade de ser Irrelevante: {:.2f}%".format(P_2*100))  

269
31
Probabilidade de ser Relevante: 89.67%
Probabilidade de ser Irrelevante: 10.33%


*Montando a função principal para o funcionamento do programa*

In [36]:
def prob(lista_de_tweets):
    prob_relevante = 0.8972
    prob_irrelevante = 0.1033
    #probabilidade da frase ser relevante
    lista_classificacao = []
    lista_classificacao_irr = []
    for frase in lista_de_tweets:
        for palavra in frase:
            soma_probir = 0.1033
            soma_prob = 0.8972
            #checa no dicionario dos relevantes
            if palavra not in drel:
                x = 1/(soma_palavras+soma_palavras_total)
                soma_prob += math.log(x)
            if palavra in drel:
                x = drel[palavra]/(soma_palavras+soma_palavras_total)
                soma_prob += math.log(x)
            #checa no dicionario dos irrelevantes
            if palavra not in dirrel:
                x = 1/(soma_palavras1+soma_palavras_total)
                
                soma_probir += math.log(x)
            if palavra in dirrel:
                x = dirrel[palavra]/(soma_palavras1+soma_palavras_total)
                soma_probir += math.log(x)
        if soma_prob > soma_probir:
             lista_classificacao.append(frase)
        
        elif soma_prob < soma_probir:
             lista_classificacao_irr.append(frase)
    k=len(lista_classificacao)
    l=len(lista_classificacao_irr)
    return ("A probabilidade de um tweet ser relevante foi de {0} e de ser irrelevante de {1}.".format((len(lista_classificacao))/(len(lista_de_tweets)),(len(lista_classificacao_irr))/(len(lista_de_tweets)))), ("Isto ocorreu pois o número de tweets relevantes encontrados pelo programa foi de {0} e o de tweets irrelevantes foi de {1}".format(k, l))

print(prob(lista))

('A probabilidade de um tweet ser relevante foi de 0.9533333333333334 e de ser irrelevante de 0.04666666666666667.', 'Isto ocorreu pois o número de tweets relevantes encontrados pelo programa foi de 286 e o de tweets irrelevantes foi de 14')


___
## Parte III - Verificando a performance

Para verificar a performance do programa, as seguintes porcentagens foram calculadas:

*(para atingir as conclusões abaixo, os 14 tweets de teste marcados como irrelevantes foram checados manualmente, e todos confirmaram de fato serem irrelevantes.)*

**Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes):**
        Dos 14 tweets marcados como não relevantes pelo programa, todos os 14 são de fato não relevantes. Desta forma, a probabilidade é de 14/14 e portanto a porcentagem de negativos verdadeiros é de **100%**.
        
**Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes):**
        286 tweets relevantes foram encontrados pelo programa, porém somente 269 realmente são realmente relevantes (e estes estão na lista de relevantes, devido ao fato da lista tweets irrelevantes de fato só conter irrelevantes). Desta forma, a probabilidade é 269/286, e portanto a porcentagem de positivos verdadeiros é de **94.1%**
        
**Porcentagem de positivos falsos (marcados como relevante mas não são relevantes):**
         269 tweets foram manualmente marcados como relevante, porém 286 tweets supostamente relevantes foram encontrados pelo programa. Desta forma, a probabilidade é 17/286 e portanto a porcentagem de positivos falsos é de **5.9%**.

**Porcentagem de negativos falsos (marcado como não relevante e são relevantes):**
        Dos 14 tweets marcados como não relevantes pelo programa, todos os 14 são de fato não relevantes. Desta forma, a probabilidade é 0/14, e portanto a porcentagem de negativos falsos é de **0%**.

*E então, o programa foi testado com o arquivo de teste*

In [39]:
Dominos_teste= pd.read_excel("tweets_Dominos.xlsx",sheet_name=1)

In [40]:
lista_teste = []
for frase in Dominos_teste.Teste:
    tweetslimposteste = removepalavra(frase)
    lista_teste.append(tweetslimposteste)
print(prob(lista_teste))

('A probabilidade de um tweet ser relevante foi de 0.955 e de ser irrelevante de 0.045.', 'Isto ocorreu pois o número de tweets relevantes encontrados pelo programa foi de 191 e o de tweets irrelevantes foi de 9')


Agora, para comparar os resultados do teste com os calculados manualmente, os quantidade de cada tweets do teste foram manualmente calculadas.

In [42]:
contador0 = Dominos_teste.Característica[Dominos_teste.Característica == 0].count()
contador1 = Dominos_teste.Característica[Dominos_teste.Característica == 1].count()
contador2 = Dominos_teste.Característica[Dominos_teste.Característica == 2].count()


contadorTotal = Dominos_teste.Característica.count()
contador01 = contador0+contador1


P_01 = (contador0+contador1)/contadorTotal 
print(contador01)
print(contador2)
P_0 = contador0/contadorTotal
P_1 = contador1/contadorTotal 
P_2 = contador2/contadorTotal

print("Probabilidade de ser Negativo: {:.2f}%".format(P_0*100))
print("Probabilidade de ser Positivo: {:.2f}%".format(P_1*100))
print("Probabilidade de ser Irrelevante: {:.2f}%".format(P_2*100))

182
18
Probabilidade de ser Negativo: 3.00%
Probabilidade de ser Positivo: 88.00%
Probabilidade de ser Irrelevante: 9.00%


*Porcentagens calculadas do arquivo de teste (comparação manual x programa), seguindo o mesmo conceitos utilizados acima.*

Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes): 9/9; **100%**

Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes): 182/191; **95.2%**
        
Porcentagem de positivos falsos (marcados como relevante mas não são relevantes): 9/191; **4.8**
      
Porcentagem de negativos falsos (marcado como não relevante e são relevantes): 0/9; **0%**

_
## Parte IV - Conclusão


Para analisar a efetividade do programa, é necessário comparar a probabilidade de tweets relevantes e irrelevantes encontrados no programa com a real probabilidade calculada manualmente. Enquanto a probabilidade encontrada foi de 95.3% de relevância, o valor real girava em torno de 89.7%. Comparando tais números numericamente, o programa teve 94.1% de precisão (89.7/95.3), o que pode ser considerado uma precisão alta e assim sendo muito adequada para o propósito do programa (de fornecer feedback para a empresa calculando a relvância dos tweets.)

Mesmo assim, o programa teve erros, como pode ser visto nos 17 tweets irrelevantes que foram marcados de fato como relevantes. Por mais não mudem muito a porcentagem de tweets relevantes (somente 17 a mais nos 269 existentes), que são os que de fato importam, mais da metade dos tweets irrelevantes foram classificados equivocadamente (17/31; 54.8%). Em uma lista de tweets com um número maior de tweets irrelevantes, o resultado poderia ser muito afetado, e a precisão desta forma seria igualmente afetada.

Buscando entender o porquê da imprecisão diante dos tweets irrelevantes, algumas hipóteses podem ser levantadas. Primeiramente, o sistema construído não é um programa exato, mas sim uma aproximação feita a partir de uma série de adaptações (como a utilização da soma de log por exemplo). Uma possível melhora envolviria não a utilização da soma de log da probabilidade, mas de fato a multiplicação da probabilidade em si que resultaria em uma exatidão maior. Além disso, uma mais ampla amostra de tweets conseguiria cada vez mais atingir uma precisão absoluta, uma vez que visando o espaço amostral tendendo ao infinito. 

Pensando em outra razão para a imprecisão, esta pode ser entendida pela suposta incapacidade do programa de ler dupla negação e sarcasmo, uma vez que a máquina não lê o sentimento expresso por um tweet, mas busca juntar dados para concluir qual sentimento aquela junção de palavras *provavelmente* expressa.

Uma vez que os grandes engenheiros André Matarazzo e Felipe Kocinas visam conquistar o planeta com sua programação, os dois possuem um plano de expandir o programa, não só classificando tweets como relevante ou não relevante mas em diversas categorias, como positivo, não positivo, delivery, gosto, etc. (visando seu contrato com a Domino's, e tais categorias que seriam feitas a partir da reaplicação do teorema de Naive Bayes).

Já com uma grande fama por seus projetos prévios, beleza e carisma, os engenheiros devem ser financiados não somente por quem são, mas especificamente por sua precisão em seus programas, sua análise reflexiva de seus erros e sua constante busca pela perfeição.